<a href="https://colab.research.google.com/github/eric-pding/RSM8415-Service-Analytics/blob/main/1_Uber_Case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case 1
## Eric Ding (MMA)




3. What is the difference between a double match rate of 2-minute wait time against a 5-minute wait time?
 - Plot the histogram and interpret the result.
 - Write one-sided hypothesis tests.
 - Is the difference statistically significant at a 95% confidence level? If you are using parametric test, you need to validate their assumptions.

4. How do the results of cancellation rate of 2-minute against a 5-minute wait time change during commute hours ? Use simple linear regression to answer the question. Use visualization to support your results.

5.  Based on the data available to you, what would you recommend doing? Would you increase the wait time from 2 to 5 minutes in the six treatment cities of the launch experiment? (explain in words)

The due date for submission is on Friday, March 1st at 11:59 am. Late submission is not accepted.

# Q1
1. Why didn’t Uber launch Express Pool with simple A/B testing for 6 cities? Why not roll out Express to 50% of users in a given city and leave the remaining 50% with the existing product offering to see how Express fares?

Answer:


#Q2
Why did Uber launch the switch-back experiment in Boston if they were already testing Express with a synthetic control lunch experiment in 12 cities?

Answer:

In [ ]:
from scipy.stats import norm

In [ ]:
mean =0.5
std_dev = 2.8
tax_rate =0.3

In [ ]:
#from table
equity = 5

In [ ]:
loss_to_wipe_equity_pre_tax = equity/(1-tax_rate)
loss_to_wipe_equity_pre_tax

7.142857142857143

In [ ]:
z_score = (-loss_to_wipe_equity_pre_tax-mean)/std_dev
probability = norm.cdf(z_score)
probability

0.0031706391958418803

In [ ]:
death_benefit = 5000000
annual_interest_rate = 0.06
probabilities = [0.011447, 0.012352, 0.013248]

# Convert annual interest rate to semiannual compounding rate
#semiannual_rate = (1 + annual_interest_rate)**(1/2) - 1
semiannual_rate = 0.03
present_value = 0
cumulative_survival_probability = 1

# Calculate present value
for i, probability_of_dying in enumerate(probabilities, start=1):

    survival_probability = cumulative_survival_probability * (1 - probability_of_dying)
    # The payout occurs in the middle of the ith year
    time_to_discount = i - 0.5
    expected_payout = death_benefit * cumulative_survival_probability * probability_of_dying
    pv = expected_payout / ((1 + semiannual_rate)**(2*time_to_discount))
    present_value += pv
    # Update the cumulative survival probability
    cumulative_survival_probability = survival_probability

# Print the semiannual rate and the corrected present value
present_value


167227.5643697698

In [ ]:
from sympy import symbols, solve

c = symbols('x')
#salary growth rate
salary_growth = 0.02

#benefit growth rate (inflation rate minus 1%)
benefit_growth = -0.01

#return on investment
r_investment_return = 0.015

# assume discount rate = 0
discount_rate = 0

employment_years = 45
retirement_years = 18
pension_percentage_of_final_salary = 0.7

#final salary
final_salary = (1 + salary_growth)**employment_years

# Calculate the present value of pension benefits
pv_pension_benefits = sum((pension_percentage_of_final_salary * final_salary * (1 + benefit_growth)**n) /
                          (1 + r_investment_return)**(n + employment_years) for n in range(retirement_years))

# Calculate the present value of contributions

pv_contributions = sum(c * (1 + salary_growth)**n / (1 + discount_rate)**n for n in range(employment_years))

# Solve for c
contribution_rate = solve(pv_contributions - pv_pension_benefits, c)

contribution_rate


[0.178356293194115]